In [9]:
import numpy as np
import pandas as pd

import random
import fileinput
import re

import matplotlib.pyplot as plt
from matplotlib.pyplot import FuncFormatter
import seaborn

In [10]:
filename_prefix = 'mortality'

In [11]:
results_filename = f'../results/{filename_prefix}_results.dta'

In [12]:
results = pd.read_stata(results_filename)

In [13]:
def get_coefficient_with_stars(row):
    return '{:.2f} [{:.2f}]'.format(row['coef'] * 100, row['stderr'] * 100) + ' ' + ('(***)' if row['pval'] <= 0.01 else '(**)' if row['pval'] <= 0.05 else '(*)' if row['pval'] < 0.1 else '')

In [16]:
results

,var,coef,stderr,pval,N,lhs,specification,r2
0,male_white_mortality:lag_mand_n_rec_00,-0.002841,0.005220,0.586249,1451,male_white,8,NaN
1,male_white_mortality:lag_mand_n_rec_01,-0.005118,0.006276,0.414754,1451,male_white,8,NaN
2,male_white_mortality:lag_mand_n_rec_02,-0.007396,0.007546,0.327079,1451,male_white,8,NaN
3,male_white_mortality:lag_mand_n_rec_03,-0.004130,0.007111,0.561374,1451,male_white,8,NaN
4,male_white_mortality:lag_mand_n_rec_04,-0.003241,0.006636,0.625239,1451,male_white,8,NaN
...,...,...,...,...,...,...,...,...
251,female_black_mortality:mand_n_rec,0.062571,0.025095,0.012656,1829,female_black,5,NaN
252,mand_n_rec,28.327488,13.223083,0.037166,1793,female_black,4,0.843929
253,mand_n_rec,50.570740,21.628845,0.023507,1793,female_black,3,0.819550
254,mand_n_rec,54.320686,21.617592,0.015248,1829,female_black,2,0.818945


In [19]:
results.query('var.str.endswith("mand_n_rec") and lhs == "female_black"', engine='python')

,var,coef,stderr,pval,N,lhs,specification,r2
248,female_black_mortality:mand_n_rec,0.038619,0.015243,0.011290,1793,female_black,8,NaN
249,female_black_mortality:mand_n_rec,0.045990,0.023028,0.045811,1793,female_black,7,NaN
250,female_black_mortality:mand_n_rec,0.050533,0.023073,0.028515,1829,female_black,6,NaN
251,female_black_mortality:mand_n_rec,0.062571,0.025095,0.012656,1829,female_black,5,NaN
252,mand_n_rec,28.327488,13.223083,0.037166,1793,female_black,4,0.843929
253,mand_n_rec,50.570740,21.628845,0.023507,1793,female_black,3,0.819550
254,mand_n_rec,54.320686,21.617592,0.015248,1829,female_black,2,0.818945
255,mand_n_rec,64.326363,23.116976,0.007588,1829,female_black,1,0.812923


In [ ]:
export = True

mask = (
    (
        (results['sub_population'].isin(['GENDER']) & results['balance_dataset'].isin([balance_dataset]))
#         | (results['sub_population'].isin(['GENDER']) & results['balance_dataset'].isin(['yes']) & results['controls'].isin(['i.year i.district_number']))
    )
    & (results['aggregation'].isin(['district_number p year']))
#     & (results['aggregation'].isin(['<none>']))
    & (results['clustering'].isin(['district_number', 'district_number_p', '<none>']))
    & (results['controls'].isin(['i.year i.district_number', 'i.district_number', 'i.year', '<none>']))
    & (results['balance_dataset'].isin([balance_dataset]))
    & (results['drop2007'] == "yes")
    & (results['drop_reorg'] == drop_reorg)
    & (results['drop_anomalies'] == drop_anomalies)
    & (results['lhs'] == "pct_mt_or_xcd") 
    & (results['new_fsa'].isin(['yes', 'no']))
    & (results['weights'].isin([weights]))
    & (results['only_prev_year_as_control'] == 'no')
    & (results['var'].isin(['p_strike_male', 'p_strike_female']))
    & actual_result
)
print(mask.sum())

result_label = 'Results by Gender'

with pd.option_context("display.max_rows", None, 'display.max_columns', None):
    table = results\
        .assign(**{
            result_label: results.apply(get_coefficient_with_stars, axis=1),
            'Specification': results.apply(get_spec_number, axis=1),
            'Sample': results['sub_population'].map({'ALL STUDENTS': 'All available', 'GENDER': 'All available', 'GENDER_COLLAPSED': 'Balanced Set of Schools'}),
            'Balancing': results['balance_dataset'].map({'yes': 'Balanced Set of Schools', 'no': 'All Available Data'}),
            'Clustering': results['clustering'].map({'<none>': 'Not Clustered', 'district_number': 'Clustered'}),
#             'Weights': results['weights'].map({' ': 'Not Weighted', '[aw=number_expected_writers]': 'Weighted'}),
            'Gender': results['var'].map({'p_strike_male': 'Male', 'p_strike_female': 'Female'}),
            'Grade': results['grade'],
            'Skill': results['skill'],
        })\
        [mask]\
        .pivot_table(
            columns=['Clustering', 'Specification'],
            index=['Gender', 'Grade', 'Skill'],
            values=[result_label],
            aggfunc={
                result_label: (lambda x: x.iloc[0] if (len(x) == 1) else np.NaN),
            }
        )
    table.style.set_properties(**{'text-align': 'left'})
    if len(table) and export:
        filename = 'Results/gender.tex'
        table.to_latex(filename, multirow=True)
        with fileinput.FileInput(filename, inplace=True) as file:
            for line_number, line in enumerate(file):
#                 if line_number == 5:
#                     print('\cline{%d-%d} \cline{%d-%d}' % (len(table.index._levels) + 1, 11, 12, 13))
                if result_label not in line and '& Sample &' not in line:
                    print(line
                          .replace('& \{\} &', '& &')
                          .replace('& Balancing &', '& &')
                          .replace('& Clustering &', '& &')
                          .replace('Gender &', ' &')
                          .replace('Female', r'\rotatebox{90}{Female}')
                          .replace('Male', r'\rotatebox{90}{Male}')
                          .replace('{l}', '{c}')
                    , end='')
    display(table)